## Курсовой проект <a class='anchor' id='hw'>

**Задание**

Реализовать сценарии поведения для диалоговой системы

- 1-ый сценарий болталка
- 2-ой погода в городе
- 3-тий праздники
- 4-ый фильмы
- 5-ый покупка товара


In [1]:
# Установка библиотеки python-telegram-bot
# !pip install python-telegram-bot==13.8
# !pip install python-telegram-bot --upgrade

In [2]:
#!pip install torch==1.8.1+cu101 torchvision==0.9.1+cu101 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
#Successfully installed pillow-10.1.0 torch-1.8.1+cu101 torchaudio-0.8.1 torchvision-0.9.1+cu101

In [3]:
#!pip install transformers
# Successfully installed filelock-3.13.1 fsspec-2023.12.2 huggingface-hub-0.19.4 safetensors-0.4.1 tokenizers-0.15.0 transformers-4.36.0

In [4]:
# Установка библиотеки диалоговых потоков. НЕ РЕГИСТРИРУЕТ РОССИЮ !!! Не использую.
#!pip install dialogflow==1.1.1

In [5]:
!pip list | findstr python-telegram-bot
!pip show google-cloud-dialogflow
!pip show dialogflow

python-telegram-bot       13.8
Name: google-cloud-dialogflow
Version: 2.26.0
Summary: Google Cloud Dialogflow API client library
Home-page: https://github.com/googleapis/google-cloud-python/tree/main/packages/google-cloud-dialogflow
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: c:\users\samol\anaconda3\envs\chat_bot\lib\site-packages
Requires: google-api-core, proto-plus, protobuf
Required-by: 
Name: dialogflow
Version: 1.1.1
Summary: Client library for the Dialogflow API
Home-page: https://github.com/googleapis/dialogflow-python-client-v2
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: c:\users\samol\anaconda3\envs\chat_bot\lib\site-packages
Requires: google-api-core
Required-by: 


1. Выберем платформу для разработки:-     будем использовать платформу для создания чат-ботов, Python с библиотекой python-telegram-boе.

In [6]:
# Импорт необходимых модулей
from telegram import Update
from telegram.ext  import Updater, CommandHandler, MessageHandler, filters, CallbackContext

import pandas as pd
import dialogflow
import requests

# Проверьте, нет ли ошибок в операторе импорта
print("Import successful.")

Import successful.


In [7]:
# 2. ПОГОДА
# Функция для получения погоды
def get_weather(city: str, OPENWEATHERMAP_API_KEY: str) -> str:
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city,
        'appid': OPENWEATHERMAP_API_KEY,
        'units': 'metric',  # You can use 'imperial' for Fahrenheit
    }

    try:
        response = requests.get(base_url, params=params)
        data = response.json()

        if response.status_code == 200:
            temperature = data['main']['temp']
            description = data['weather'][0]['description']
            return f'Текущая температура в {city}: {temperature}°C, {description}'
        else:
            return f'Ошибка: {data["message"]}'

    except Exception as e:
        print(f'Error: {e}')
        return ''


# Функция для обработки запроса о погоде
def handle_weather_request(update: Update, context: CallbackContext) -> None:
    city = context.args[0] if context.args else 'Moscow'  # Default city is Moscow if not provided
    OPENWEATHERMAP_API_KEY = context.bot_data.get('OPENWEATHERMAP_API_KEY')

    if not OPENWEATHERMAP_API_KEY:
        update.message.reply_text('Ошибка: API-ключ для OpenWeatherMap не найден.')
        return

    weather_data = get_weather(city, OPENWEATHERMAP_API_KEY)

    if weather_data:
        update.message.reply_text(weather_data)
    else:
        update.message.reply_text('Не удалось получить информацию о погоде.')


In [8]:
# 3.  ПРАЗДНИКИ
# Функция для получения праздников
def get_holidays(year: int, CALENDARIFIC_API_KEY: str) -> str:
    base_url = "https://calendarific.com/api/v2/holidays"
    params = {
        'api_key': CALENDARIFIC_API_KEY,
        'country': 'RU',  # Country code for Russia
        'year': year,
    }

    try:
        response = requests.get(base_url, params=params)
        data = response.json()

        if response.status_code == 200:
            holidays_list = [holiday['name'] for holiday in data['response']['holidays']]
            return f'Праздники в России в {year}: {", ".join(holidays_list)}'
        else:
            return f'Ошибка: {data["meta"]["error_detail"]}'

    except Exception as e:
        print(f'Error: {e}')
        return ''

# Функция для обработки запроса о праздниках
def handle_holidays_request(update: Update, context: CallbackContext) -> None:
    year = context.args[0] if context.args else 2023  # Default to the current year if not provided
    CALENDARIFIC_API_KEY = context.bot_data.get('CALENDARIFIC_API_KEY')

    if not CALENDARIFIC_API_KEY:
        update.message.reply_text('Ошибка: API-ключ для Calendarific не найден.')
        return

    holidays_data = get_holidays(year, CALENDARIFIC_API_KEY)

    if holidays_data:
        update.message.reply_text(holidays_data)
    else:
        update.message.reply_text('Не удалось получить информацию о праздниках.')


In [9]:
#  4. КИНО
# Функция для получения информации о фильме из OMDB
def get_movie_info_omdb(movie_title: str, OMDB_API_KEY: str) -> str:
    base_url = "http://www.omdbapi.com/"
    params = {
        'apikey': OMDB_API_KEY,
        't': movie_title,
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()

        data = response.json()

        if response.status_code == 200 and data.get('Title'):
            title = data['Title']
            description = data['Plot']
            release_date = data['Year']
            return f"Название: {title}\nОписание: {description}\nГод выпуска: {release_date}"
        else:
            return "Информация не найдена на OMDb."

    except requests.RequestException as e:
        print(f"Ошибка: {e}")
        return "Не удалось получить информацию о фильме."

# Функция для обработки запроса о фильме
def handle_movies_request(update: Update, context: CallbackContext) -> None:
    movie_title = ' '.join(context.args) if context.args else 'Interstellar'
    OMDB_API_KEY = context.bot_data.get('OMDB_API_KEY')

    if not OMDB_API_KEY:
        update.message.reply_text('Ошибка: API-ключ для OMDB не найден.')
        return

    movie_info = get_movie_info_omdb(movie_title, OMDB_API_KEY)
    update.message.reply_text(movie_info)


In [10]:
#  5. ПОКУПКИ
# Функция для обработки запроса о покупках
def handle_shopping_request(update: Update, context: CallbackContext) -> None:
    query = context.args
    if not query:
        update.message.reply_text("Пожалуйста, укажите запрос для поиска товаров.")
        return

    query = " ".join(query).lower()
    results = shop_data[shop_data['title'].str.lower().str.contains(query, na=False) | shop_data['description'].str.lower().str.contains(query, na=False)]

    if results.empty:
        update.message.reply_text("По вашему запросу ничего не найдено.")
    else:
        for index, row in results.iterrows():
            response = f"Название: {row['title']}\nОписание: {row['descrirption']}\nID товара: {row['product_id']}\nКатегория: {row['category_id']}\nПодкатегория: {row['subcategory_id']}\nСсылка на изображение: {row['image_links']}"
            update.message.reply_text(response)



In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Загрузка предобученной модели и токенизатора
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

# Функция для генерации ответа от модели
def generate_response(prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids, max_length=max_length, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

C:\Users\SAMOL\anaconda3\envs\Chat_bot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 720/720 [00:00<00:00, 360kB/s]
pytorch_model.bin: 100%|██████████| 551M/551M [00:48<00:00, 11.4MB/s] 
tokenizer_config.json: 100%|██████████| 1.25k/1.25k [00:00<00:00, 419kB/s]
vocab.json: 100%|██████████| 1.71M/1.71M [00:00<00:00, 3.87MB/s]
merges.txt: 100%|██████████| 1.27M/1.27M [00:00<00:00, 7.17MB/s]
special_tokens_map.json: 100%|██████████| 574/574 [00:00<00:00, 286kB/s]
C:\Users\SAMOL\anaconda3\envs\Chat_bot\lib\site-packages\transformers\generation\configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Привет, как дела?

— Нормально, — ответил я.
		Я не знал, что сказать. Я не хотел, чтобы кто-то знал обо мне больше, чем я сам. Но я не мог не думать об этом, потому что это было так важно для меня, и я хотел знать, почему я так беспокоюсь о тебе. И я сказал себе: «Я знаю, о чем ты думаешь, но не могу сказать тебе правду». Я сказал:


In [39]:
# 1. Токены для Телеграмм бота.
PATH_TOKEN = './token.txt'
# токен для Телеграма
with open(PATH_TOKEN, 'r') as f1:
  TOKEN = f1.read()

# 2. Токен для Погода в городе.
PATH_OPENWEATHERMAP_API_KEY = './API_openweather.txt'
with open(PATH_OPENWEATHERMAP_API_KEY, 'r') as f2:
  OPENWEATHERMAP_API_KEY = f2.read()

# 3. Токен для календарного API
PATH_CALENDARIFIC_API_KEY = './API_calendar.txt'
with open(PATH_CALENDARIFIC_API_KEY, 'r') as f3:
    CALENDARIFIC_API_KEY = f3.read()

# 4. API Key for OMDB
PATH_OMDB_API_KEY = './API_OMDB.txt'
with open(PATH_OMDB_API_KEY, 'r') as f4:
    OMDB_API_KEY = f4.read()


2. Создадим основной код бота:-      Используя выбранную платформу, напишим код для обработки команд и сообщений от пользователя.

In [ ]:
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd
import re
import time
import requests

# Загрузка предобученной модели и токенизатора
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

# Загрузим базу данных продуктов
shop_data = pd.read_csv("ProductsDataset.csv")


# Функция для обработки команды /start
def start(update: Update, context: CallbackContext) -> None:
    update.message.reply_text("Привет! Я могу рассказать о погоде, праздниках, кино и не только. Просто напиши мне что-нибудь.")


# Функция для обработки запросов о покупках
def handle_shopping_request(update: Update, context: CallbackContext) -> None:
    user_input = update.message.text

    # Ищем ключевые слова в сообщении пользователя
    keywords = ['купить', 'покупки', 'покупку', 'покупка']
    if any(keyword in user_input.lower() for keyword in keywords):
        # Извлекаем название товара из запроса
        match = re.search(r'\b(?:купить|покупки|покупку|покупка)\s+(.+)', user_input, re.IGNORECASE)
        product_name = match.group(1) if match else None

        if product_name:
            # Ищем товар в базе данных
            results = shop_data[shop_data['title'].str.lower().str.contains(product_name, na=False)]

            if not results.empty:
                # Отправляем информацию о найденных товарах пользователю
                response = "Найденные товары:\n"
                for index, row in results.iterrows():
                    response += (
                        f"Название: {row['title']}\nОписание: {row['descrirption']}\n"
                        f"ID товара: {row['product_id']}\nКатегория: {row['category_id']}\n"
                        f"Подкатегория: {row['subcategory_id']}\n"
                        f"Ссылка на изображение: {row['image_links']}\n\n"
                    )

                update.message.reply_text(response)
            else:
                update.message.reply_text("По вашему запросу ничего не найдено.")
        else:
            update.message.reply_text("Пожалуйста, укажите название товара для поиска.")
    else:
        update.message.reply_text("Извините, я не могу обработать ваш запрос о покупках.")


# Функция для получения погоды
def get_weather(city: str) -> str:
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city,
        'appid': OPENWEATHERMAP_API_KEY,
        'units': 'metric',
    }

    try:
        response = requests.get(base_url, params=params)
        data = response.json()

        if response.status_code == 200:
            temperature = data['main']['temp']
            description = data['weather'][0]['description']
            return f'Текущая температура в {city}: {temperature}°C, {description}'
        else:
            return f'Ошибка: {data["message"]}'
    except Exception as e:
        print(f'Error: {e}')
        return ''

# Функция для получения праздников
def get_holidays(year: int) -> str:
    base_url = "https://calendarific.com/api/v2/holidays"
    params = {
        'api_key': CALENDARIFIC_API_KEY,
        'country': 'RU',
        'year': year,
    }

    try:
        response = requests.get(base_url, params=params)
        data = response.json()

        if response.status_code == 200:
            holidays_list = [holiday['name'] for holiday in data['response']['holidays']]
            return f'Праздники в России в {year}: {", ".join(holidays_list)}'
        else:
            return f'Ошибка: {data["meta"]["error_detail"]}'
    except Exception as e:
        print(f'Error: {e}')
        return ''

# Функция для получения информации о фильме из OMDB
def get_movie_info_omdb(movie_title: str) -> str:
    base_url = "http://www.omdbapi.com/"
    params = {
        'apikey': OMDB_API_KEY,
        't': movie_title,
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()

        data = response.json()

        if response.status_code == 200 and data.get('Title'):
            title = data['Title']
            description = data['Plot']
            release_date = data['Year']
            return f"Название: {title}\nОписание: {description}\nГод выпуска: {release_date}"
        else:
            return "Информация не найдена на OMDb."

    except requests.RequestException as e:
        print(f"Ошибка: {e}")
        return "Не удалось получить информацию о фильме."

# Функция для обработки текстовых сообщений
def handle_text(update: Update, context: CallbackContext) -> None:
    user_input = update.message.text

    # Проверяем, содержит ли сообщение запрос о погоде
    if re.search(r'\bкакая погода в городе\b', user_input, re.IGNORECASE):
        # Извлекаем город из запроса
        match = re.search(r'\bгороде\s+(\w+)', user_input, re.IGNORECASE)
        city = match.group(1) if match else 'Moscow'  # По умолчанию город - Moscow

        # Вызываем функцию для получения погоды и отправляем ответ пользователю
        weather_data = get_weather(city)
        update.message.reply_text(weather_data)
    # Проверяем, содержит ли сообщение запрос о праздниках
    elif re.search(r'\bкакие праздники в (\d{4})\b', user_input, re.IGNORECASE):
        # Извлекаем год из запроса
        match = re.search(r'\bпраздники в (\d{4})\b', user_input, re.IGNORECASE)
        year = int(match.group(1)) if match else 2023  # По умолчанию год - 2023

        # Вызываем функцию для получения праздников и отправляем ответ пользователю
        holidays_data = get_holidays(year)
        update.message.reply_text(holidays_data)
    # Проверяем, содержит ли сообщение запрос о фильме
    elif re.search(r'\bрасскажи о фильме\b', user_input, re.IGNORECASE):
        # Извлекаем название фильма из запроса
        match = re.search(r'\bфильме\s+(.+)', user_input, re.IGNORECASE)
        movie_title = match.group(1) if match else 'Interstellar'  # По умолчанию фильм - Interstellar

        # Вызываем функцию для получения информации о фильме и отправляем ответ пользователю
        movie_info = get_movie_info_omdb(movie_title)
        update.message.reply_text(movie_info)
    # Проверяем, содержит ли сообщение запрос о покупках
    elif re.search(r'\b(?:купить|покупки|покупку|покупка)\b', user_input, re.IGNORECASE):
        handle_shopping_request(update, context)
    else:
        # В противном случае, генерируем ответ с использованием GPT-3
        bot_response = generate_response(user_input)
        update.message.reply_text(bot_response)


# функция main()
def main() -> None:
    updater = Updater(TOKEN)
    dp = updater.dispatcher

    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("search", search))
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_text))
    dp.add_handler(MessageHandler(Filters.text & Filters.regex(r'\b(?:купить|покупки|покупку|покупка)\b'), handle_shopping_request))

    while True:
        try:
            updater.start_polling()
            updater.idle()
        except Exception as e:
            print(f"Error: {e}")
            print("Restarting in 10 seconds...")
            time.sleep(10)

if __name__ == '__main__':
    main()